# Neural Network Example

In [1]:
import tensorflow as tf

In [62]:
# Parameters
learning_rate = 0.01
num_steps = 50000
batch_size = 20000
display_step = 1000

n_PC = 40
# Network Parameters
n_hidden_1 = n_PC-2 # 1st layer number of neurons
n_hidden_2 = n_PC-4 # 2nd layer number of neurons
num_input = n_PC # data input # of PC
num_classes = 12 # total classes 

# tf Graph input
x = tf.placeholder("float", shape = [None, num_input])
y = tf.placeholder("float", shape = [None, num_classes])

In [63]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [64]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [65]:
# Construct model
logits = neural_net(x)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [66]:
import pandas as pd
df = pd.read_csv('[FOR_DISTRIBUTE]train_data.csv')
X = df[df.columns[3:]].values
Y = df['Class']
import numpy as np
Y_map = dict()
n = 0
for i in set(Y):
    Y_map[i] = n
    n+=1
Y2 = []
for i in Y:
    Y2.append(Y_map[i])
Y2 = np.asarray(Y2)
temp = np.zeros(12)
# Y2
Y3 = []
for i in Y2:
    temp = np.zeros(12)
    temp[i]+=1.
    Y3.append(temp)
Y3 = np.asarray(Y3)

from sklearn import decomposition

X = df[df.columns[3:]].values
pca = decomposition.PCA(n_components = n_PC)
pca.fit(X)
X = pca.transform(X) # X * Maty

from sklearn.cross_validation import train_test_split
test_size = 0.5
X_train, X_test, y_train, y_test = train_test_split(X, Y3,test_size=test_size)
bs_s = 0
bs_e = batch_size

In [67]:
# Start training

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = X_train[bs_s:bs_e], y_train[bs_s:bs_e]
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy with TRAIN SET
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x,
                                                                 y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            # Calculate accuracy with TEST SET
            print("Testing Accuracy:", \
                sess.run(accuracy, feed_dict={x: X_test[bs_s:bs_e],
                                              y: y_test[bs_s:bs_e]}))

    print("Optimization Finished!")

    

Step 1, Minibatch Loss= 181.9934, Training Accuracy= 0.118
Testing Accuracy: 0.11365
Step 1000, Minibatch Loss= 1.2940, Training Accuracy= 0.574
Testing Accuracy: 0.56345
Step 2000, Minibatch Loss= 1.2744, Training Accuracy= 0.579
Testing Accuracy: 0.56645
Step 3000, Minibatch Loss= 1.2701, Training Accuracy= 0.577
Testing Accuracy: 0.5677
Step 4000, Minibatch Loss= 1.2082, Training Accuracy= 0.593
Testing Accuracy: 0.5796
Step 5000, Minibatch Loss= 1.2471, Training Accuracy= 0.585
Testing Accuracy: 0.57215
Step 6000, Minibatch Loss= 1.2323, Training Accuracy= 0.586
Testing Accuracy: 0.57325
Step 7000, Minibatch Loss= 1.2254, Training Accuracy= 0.582
Testing Accuracy: 0.57065
Step 8000, Minibatch Loss= 1.2317, Training Accuracy= 0.580
Testing Accuracy: 0.5644
Step 9000, Minibatch Loss= 1.2469, Training Accuracy= 0.575
Testing Accuracy: 0.562
Step 10000, Minibatch Loss= 1.2195, Training Accuracy= 0.588
Testing Accuracy: 0.5732
Step 11000, Minibatch Loss= 1.2236, Training Accuracy= 0.596